In [1]:
import pandas as pd
from datetime import datetime
from gensim.models import FastText
from tqdm import tqdm
from soynlp.hangle import compose, decompose, character_is_korean
import re
import gensim


In [5]:
# 자모를 분해한 파일 생성하기1(말뭉치(corpus) 데이터 불러오기)

nouns = []
with open('corpus_nouns.txt', 'r', encoding='utf-8') as f:
    for sentence in tqdm(f.readlines()):
        nouns.append(sentence.split())

# morphs = []
# with open('/storage/rok/Korean_Embedding/corpus_morphs.txt', 'r') as f:
#     for sentence in tqdm(f.readlines()):
#         morphs.append(sentence.split())

100%|███████████████████████████████████████████████████████████████████████| 844013/844013 [00:27<00:00, 30429.37it/s]


In [5]:
del nouns
del morphs

In [2]:
doublespace_pattern = re.compile('\s+')
def jamo_sentence(sent):

    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
        return cjj_

    sent_ = []
    for char in sent:
        if character_is_korean(char):
            sent_.append(transform(char))
        else:
            sent_.append(char)
    sent_ = doublespace_pattern.sub(' ', ''.join(sent_))
    return sent_



In [3]:
def jamo_to_word(jamo):
    jamo_list, idx = [], 0
    while idx < len(jamo):
        if not character_is_korean(jamo[idx]):
            jamo_list.append(jamo[idx])
            idx += 1
        else:
            jamo_list.append(jamo[idx:idx+3])
            idx += 3
    word = ""
    for jamo_char in jamo_list:
        if len(jamo_char) == 1:
            word += jamo_char
        elif jamo_char[2] == "-":
            word += compose(jamo_char[0], jamo_char[1], " ")
        else:
            word += compose(jamo_char[0], jamo_char[1], jamo_char[2])
    return word

In [4]:
def process_jamo(tokenized_corpus_fname, output_fname):
    with open(tokenized_corpus_fname, 'r', encoding='utf-8') as f1, \
            open(output_fname, 'w', encoding='utf-8') as f2:
        for line in tqdm(f1):
            sentence = line.replace('\n', '').strip()
            processed_sentence = jamo_sentence(sentence)
            f2.writelines(processed_sentence + '\n')

In [5]:
def transform(list):
    return [(jamo_to_word(w), r) for(w,r) in list]

In [10]:
# 자모를 분해한 파일 생성하기2 (자모 분해하기: 30분 소요)

process_jamo(tokenized_corpus_fname='corpus_nouns.txt', output_fname='corpus_nouns_jamo.txt')

# process_jamo(tokenized_corpus_fname='corpus_morphs.txt', output_fname='corpus_morphs_jamo.txt')

844013it [17:57, 783.53it/s] 


In [ ]:
del nouns

In [12]:
corpus_morphs = [i.strip().split(" ") for i in tqdm(open('/storage/rok/Korean_Embedding/corpus_morphs_jamo.txt', 'r', encoding='utf-8').readlines())]

100%|██████████| 844013/844013 [00:33<00:00, 25374.98it/s]


In [6]:
corpus_nouns = [i.strip().split(" ") for i in tqdm(open('corpus_nouns_jamo.txt', 'r', encoding='utf-8').readlines())]

100%|████████████████████████████████████████████████████████████████████████| 844013/844013 [01:29<00:00, 9428.24it/s]


#### using skip-gram 

In [22]:
begin=datetime.now()

In [23]:
fasttext_morphs = FastText(corpus_morphs, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [24]:
end=datetime.now() - begin

In [25]:
print(end)

1:08:04.985114


In [26]:
transform(fasttext_morphs.wv.most_similar(jamo_sentence('코로나'), topn=10))

[('감염증', 0.9166404604911804),
 ('신종', 0.8719736933708191),
 ('바이러스', 0.8613077402114868),
 ('대유행', 0.8116739988327026),
 ('코로나도', 0.7756314277648926),
 ('사이러스', 0.7590113282203674),
 ('확산', 0.7508911490440369),
 ('바이러', 0.7111417651176453),
 ('방역', 0.7084428071975708),
 ('팬데', 0.7076288461685181)]

In [27]:
fasttext_morphs.save('/storage/rok/Korean_Embedding/embedding_models/FastText_morphs')

In [8]:
begin=datetime.now()

In [11]:
fasttext_nouns = FastText(corpus_nouns, size=100, window=5, min_count=5, workers=4, sg=1)

In [12]:
end=datetime.now() - begin

In [13]:
print(end)

0:46:06.539925


In [14]:
transform(fasttext_nouns.wv.most_similar(jamo_sentence('코로나'), topn=10))

[('감염증', 0.934538722038269),
 ('바이러스', 0.8809232711791992),
 ('신종', 0.8769291639328003),
 ('대유행', 0.8149067163467407),
 ('확산', 0.7848906517028809),
 ('사이러스', 0.7675048112869263),
 ('바이러', 0.7502391934394836),
 ('코로나도', 0.7405038475990295),
 ('확진', 0.7358541488647461),
 ('여파', 0.7154252529144287)]

In [15]:
fasttext_nouns.save('FastText_nouns')

In [16]:
del corpus_nouns, fasttext_nouns

In [ ]:
m0 = gensim.models.FastText.load('FastText_morphs')
transform(m0.wv.most_similar(jamo_sentence('코로나'), topn=10))

In [36]:
m1 = gensim.models.FastText.load('FastText_nouns')
transform(m1.wv.most_similar(jamo_sentence('코로나'), topn=10))

[('감염증', 0.9374035000801086),
 ('신종', 0.8863914608955383),
 ('바이러스', 0.8838632702827454),
 ('대유행', 0.813446581363678),
 ('확산', 0.7794215679168701),
 ('사이러스', 0.7732242345809937),
 ('바이러', 0.7599300742149353),
 ('코로나도', 0.752407968044281),
 ('확진', 0.7384893894195557),
 ('여파', 0.7216395139694214)]

In [17]:
m1 = gensim.models.FastText.load('FastText_nouns')
transform(m1.wv.most_similar(jamo_sentence('코로나'), topn=10))

[('감염증', 0.934538722038269),
 ('바이러스', 0.8809232711791992),
 ('신종', 0.8769291639328003),
 ('대유행', 0.8149067163467407),
 ('확산', 0.7848906517028809),
 ('사이러스', 0.7675048112869263),
 ('바이러', 0.7502391934394836),
 ('코로나도', 0.7405038475990295),
 ('확진', 0.7358541488647461),
 ('여파', 0.7154252529144287)]

In [24]:
m1 = gensim.models.FastText.load('FastText_nouns')
transform(m1.wv.most_similar(jamo_sentence('코로나'), topn=10))

[('감염증', 0.934538722038269),
 ('바이러스', 0.8809232711791992),
 ('신종', 0.8769291639328003),
 ('대유행', 0.8149067163467407),
 ('확산', 0.7848906517028809),
 ('사이러스', 0.7675048112869263),
 ('바이러', 0.7502391934394836),
 ('코로나도', 0.7405038475990295),
 ('확진', 0.7358541488647461),
 ('여파', 0.7154252529144287)]

In [18]:
del m1

###  w2v

In [2]:
from gensim.models import Word2Vec
import gensim

from datetime import datetime
from tqdm import tqdm
import pandas as pd
import os


In [3]:
nouns = []
with open('corpus_nouns.txt', 'r', encoding='utf-8') as f:
    for sentence in tqdm(f.readlines()):
        nouns.append(sentence.split())

100%|███████████████████████████████████████████████████████████████████████| 844013/844013 [00:18<00:00, 44727.56it/s]


In [4]:
def w2v(data, sg, name):
    
    begin = datetime.now()
    model = Word2Vec(sentences=data, size=100, window=5, min_count=5, workers=4, sg=sg)
    end = datetime.now() - begin
    print(end)
        
#     if sg == 0:    
#         model.save('CBOW_'+name)
#     elif sg == 1:
#         model.save('Skip_gram_'+name)
    
    return model

In [5]:
skipgram_nouns = w2v(data=nouns, sg=1, name='nouns')
skipgram_nouns.wv.most_similar('코로나', topn=10)

0:20:26.536875


[('감염증', 0.9315603971481323),
 ('신종', 0.8666401505470276),
 ('바이러스', 0.8529074192047119),
 ('대유행', 0.7721904516220093),
 ('확산', 0.7620081901550293),
 ('전별', 0.7346984148025513),
 ('점종', 0.7079029679298401),
 ('확진', 0.7045122385025024),
 ('다립', 0.6960322260856628),
 ('역선', 0.6932239532470703)]

In [6]:
skipgram_nouns.save('Skip_gram_nouns')

In [7]:
m1 = gensim.models.Word2Vec.load('Skip_gram_nouns')

In [8]:
m1.wv.most_similar('코로나', topn=10)

[('감염증', 0.9315603971481323),
 ('신종', 0.8666401505470276),
 ('바이러스', 0.8529074192047119),
 ('대유행', 0.7721904516220093),
 ('확산', 0.7620081901550293),
 ('전별', 0.7346984148025513),
 ('점종', 0.7079029679298401),
 ('확진', 0.7045122385025024),
 ('다립', 0.6960322260856628),
 ('역선', 0.6932239532470703)]

In [33]:
m1.wv.most_similar('코로나', topn=10)

[('ㅇㅓㅁㅍㅖ-ㅁㅜㄹ', 0.3091914653778076),
 ('ㅅㅜ-ㄹㅠㄱ', 0.30277109146118164),
 ('ㅎㅓㄴㅌㅣㅇㅋㅐㅂ', 0.2951756417751312),
 ('ㅅㅡ-ㅌㅗㄱㅎㅗㄹㄹㅡㅁ', 0.2912899851799011),
 ('ㅌㅓ-ㅂㅜㄹ', 0.28863829374313354),
 ('ㅅㅏㄴㅂㅏㅌ', 0.2860708236694336),
 ('ㅍㅣ-ㅍㅡ-ㅌㅣㄴ', 0.2818150222301483),
 ('ㄹㅏ-ㅌㅜ-ㄹㅡ-', 0.2814582288265228),
 ('ㅂㅣ-ㅈㅔㄴ', 0.2802598774433136),
 ('ㅅㅏㄴㅂㅏㄹ', 0.27986812591552734)]

In [9]:
del nouns, skipgram_nouns, m1

### LSA

In [13]:
from gensim.models import LsiModel
from gensim import corpora

In [10]:
nouns=[]
with open('corpus_nouns.txt', 'r', encoding='utf-8') as f:
    for sentence in tqdm(f.readlines()):
        nouns.append(sentence.split(' '))

100%|███████████████████████████████████████████████████████████████████████| 844013/844013 [00:16<00:00, 49667.44it/s]


In [11]:
# tokenized_doc_morphs = pd.Series(morphs)
tokenized_doc_nouns = pd.Series(nouns)

In [14]:
# dictionary_morphs = corpora.Dictionary(tokenized_doc_morphs)
dictionary_nouns = corpora.Dictionary(tokenized_doc_nouns)

In [15]:
import numpy as np

# Save
# np.save('/storage/rok/Korean_Embedding/LSA_dict_morphs.npy', dictionary_morphs.token2id)
np.save('LSA_dict_nouns.npy', dictionary_nouns.token2id)

# Load
# d1 = np.load('/storage/rok/Korean_Embedding/LSA_dict_morphs.npy',allow_pickle='TRUE').item()
d2 = np.load('LSA_dict_nouns.npy',allow_pickle='TRUE').item()

# print(d1['사랑']) # displays "world"

In [ ]:
del nouns

In [16]:
doc_term_matrix_nouns = [dictionary_nouns.doc2bow(doc) for doc in tokenized_doc_nouns]
lsamodel_nouns = LsiModel(doc_term_matrix_nouns, num_topics=10, id2word = dictionary_nouns)  # train model
lsamodel_nouns.print_topics(num_topics=10, num_words=10)



[(0,
  '0.220*"코로나" + 0.151*"확진" + 0.149*"접종" + 0.143*"대표" + 0.135*"국민" + 0.131*"기업" + 0.129*"서울" + 0.127*"정부" + 0.127*"백신" + 0.125*"지역"'),
 (1,
  '-0.444*"접종" + -0.360*"확진" + -0.329*"백신" + -0.267*"코로나" + 0.173*"대표" + -0.136*"감염" + 0.136*"기업" + -0.131*"방역" + -0.121*"수도" + 0.115*"국민"'),
 (2,
  '-0.297*"국민" + 0.219*"기업" + -0.219*"후보" + 0.214*"사업" + -0.213*"총장" + -0.203*"대표" + -0.169*"의원" + -0.160*"지사" + -0.142*"대선" + -0.141*"대통령"'),
 (3,
  '0.540*"접종" + -0.426*"확진" + 0.410*"백신" + 0.135*"예약" + -0.131*"올림픽" + -0.129*"수도" + -0.111*"코로나" + -0.109*"단계" + -0.107*"검사" + -0.103*"서울"'),
 (4,
  '-0.495*"올림픽" + -0.330*"도쿄" + -0.281*"선수" + -0.255*"일본" + -0.232*"한국" + -0.141*"경기" + 0.122*"국민" + -0.121*"중국" + -0.111*"미국" + -0.109*"여자"'),
 (5,
  '-0.232*"지원" + 0.217*"투자" + 0.213*"금융" + -0.210*"사업" + 0.193*"분기" + -0.166*"지역" + -0.150*"교육" + 0.150*"시장" + 0.137*"대비" + 0.134*"증가"'),
 (6,
  '0.280*"수사" + -0.237*"국민" + 0.217*"사건" + -0.178*"소득" + 0.177*"검찰" + -0.165*"정부" + 0.163*"검사" + -0.147*"지원" + -0.144*"지

#### 밑에 틀렷으니 다시 실행만하면됨

In [17]:
lsamodel_nouns.save('LSA_nouns')

In [25]:
del  lsamodel_nouns